# Training a NN for metric on CICY with homog

## Import the required packages/functions

In [1]:
#disable GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import gc
import sys
import os
import re
import logging
import pickle
import sys
#sys.path.append("/Users/kit/Documents/Phys_Working/MF metric")
#sys.path.append("/home/f/fraser-talientec/PhysicalYukawas")
sys.path.append("/home/f/fraser-talientec/cymetricap/AlphaPrime")

logging.basicConfig(stream=sys.stdout)

from cymetric.pointgen.pointgen import PointGenerator
from cymetric.pointgen.nphelper import prepare_dataset, prepare_basis_pickle

import tensorflow as tf
import tensorflow.keras as tfk

tf.get_logger().setLevel('ERROR')


from cymetric.models.tfmodels import PhiFSModel, MultFSModel, FreeModel, MatrixFSModel, AddFSModel, PhiFSModelToric, MatrixFSModelToric
from cymetric.models.tfhelper import prepare_tf_basis, train_model
from cymetric.models.callbacks import SigmaCallback, KaehlerCallback, TransitionCallback, RicciCallback, VolkCallback, AlphaCallback
from cymetric.models.metrics import SigmaLoss, KaehlerLoss, TransitionLoss, RicciLoss, VolkLoss, TotalLoss

#from NewCustomMetrics import *
from laplacian_funcs import *
#from generate_and_train_all_nnsHOLO import *
from custom_networks import *
import sys
import importlib
from AlphaPrimeModel import *
#reimport ALphaPrimeModel
importlib.reload(sys.modules['AlphaPrimeModel'])
from AlphaPrimeModel import *
#reimport custom_networks
importlib.reload(sys.modules['custom_networks'])
from custom_networks import *




2024-08-12 14:25:10.668634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 14:25:14.837995: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-12 14:25:14.838031: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: tplxdt127
2024-08-12 14:25:14.838040: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: tplxdt127
2024-08-12 14:25:14.838106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 555.42.6
2024-08-12 14:25:14.838131: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:2

## Point Cloud Generation

Set the properties of the defining polynomial. And the point in Kahler Moduli space

If correct, this should be for the following defining polynomial
$0.44 x_{1,0}^2 x_{3,0}^2 x_{4,0}^2 x_{2,0}^2+0.88 x_{1,1}^2 x_{3,0}^2 x_{4,0}^2 x_{2,0}^2+0.88 x_{1,0}^2 x_{3,1}^2 x_{4,0}^2 x_{2,0}^2+0.88 x_{1,1}^2 x_{3,1}^2 x_{4,0}^2 x_{2,0}^2-0.03 x_{1,0} x_{1,1} x_{3,0} x_{3,1} x_{4,0}^2 x_{2,0}^2+0.44 x_{1,0}^2 x_{3,0}^2 x_{4,1}^2 x_{2,0}^2+0.44 x_{1,1}^2 x_{3,0}^2 x_{4,1}^2 x_{2,0}^2+0.88 x_{1,0}^2 x_{3,1}^2 x_{4,1}^2 x_{2,0}^2+0.44 x_{1,1}^2 x_{3,1}^2 x_{4,1}^2 x_{2,0}^2-0.41 x_{1,0} x_{1,1} x_{3,0} x_{3,1} x_{4,1}^2 x_{2,0}^2-0.41 x_{1,0} x_{1,1} x_{3,0}^2 x_{4,0} x_{4,1} x_{2,0}^2-0.03 x_{1,0} x_{1,1} x_{3,1}^2 x_{4,0} x_{4,1} x_{2,0}^2+0.62 x_{1,0}^2 x_{3,0} x_{3,1} x_{4,0} x_{4,1} x_{2,0}^2+0.62 x_{1,1}^2 x_{3,0} x_{3,1} x_{4,0} x_{4,1} x_{2,0}^2-0.62 x_{1,0} x_{1,1} x_{2,1} x_{3,0}^2 x_{4,0}^2 x_{2,0}-0.62 x_{1,0} x_{1,1} x_{2,1} x_{3,1}^2 x_{4,0}^2 x_{2,0}+0.41 x_{1,0}^2 x_{2,1} x_{3,0} x_{3,1} x_{4,0}^2 x_{2,0}+0.03 x_{1,1}^2 x_{2,1} x_{3,0} x_{3,1} x_{4,0}^2 x_{2,0}-0.62 x_{1,0} x_{1,1} x_{2,1} x_{3,0}^2 x_{4,1}^2 x_{2,0}-0.62 x_{1,0} x_{1,1} x_{2,1} x_{3,1}^2 x_{4,1}^2 x_{2,0}+0.03 x_{1,0}^2 x_{2,1} x_{3,0} x_{3,1} x_{4,1}^2 x_{2,0}+0.41 x_{1,1}^2 x_{2,1} x_{3,0} x_{3,1} x_{4,1}^2 x_{2,0}+0.41 x_{1,0}^2 x_{2,1} x_{3,0}^2 x_{4,0} x_{4,1} x_{2,0}+0.03 x_{1,1}^2 x_{2,1} x_{3,0}^2 x_{4,0} x_{4,1} x_{2,0}+0.03 x_{1,0}^2 x_{2,1} x_{3,1}^2 x_{4,0} x_{4,1} x_{2,0}+0.41 x_{1,1}^2 x_{2,1} x_{3,1}^2 x_{4,0} x_{4,1} x_{2,0}+0.9 x_{1,0} x_{1,1} x_{2,1} x_{3,0} x_{3,1} x_{4,0} x_{4,1} x_{2,0}+0.44 x_{1,0}^2 x_{2,1}^2 x_{3,0}^2 x_{4,0}^2+0.88 x_{1,1}^2 x_{2,1}^2 x_{3,0}^2 x_{4,0}^2+0.44 x_{1,0}^2 x_{2,1}^2 x_{3,1}^2 x_{4,0}^2+0.44 x_{1,1}^2 x_{2,1}^2 x_{3,1}^2 x_{4,0}^2-0.41 x_{1,0} x_{1,1} x_{2,1}^2 x_{3,0} x_{3,1} x_{4,0}^2+0.88 x_{1,0}^2 x_{2,1}^2 x_{3,0}^2 x_{4,1}^2+0.88 x_{1,1}^2 x_{2,1}^2 x_{3,0}^2 x_{4,1}^2+0.88 x_{1,0}^2 x_{2,1}^2 x_{3,1}^2 x_{4,1}^2+0.44 x_{1,1}^2 x_{2,1}^2 x_{3,1}^2 x_{4,1}^2-0.03 x_{1,0} x_{1,1} x_{2,1}^2 x_{3,0} x_{3,1} x_{4,1}^2-0.03 x_{1,0} x_{1,1} x_{2,1}^2 x_{3,0}^2 x_{4,0} x_{4,1}-0.41 x_{1,0} x_{1,1} x_{2,1}^2 x_{3,1}^2 x_{4,0} x_{4,1}+0.62 x_{1,0}^2 x_{2,1}^2 x_{3,0} x_{3,1} x_{4,0} x_{4,1}+0.62 x_{1,1}^2 x_{2,1}^2 x_{3,0} x_{3,1} x_{4,0} x_{4,1}$

In [7]:
monomialsTQ = 5*np.eye(5, dtype=np.int64)
coefficientsTQ = np.ones(5)
kmoduliTQ = np.ones(1)
ambientTQ = np.array([4])
nameofmanifold="Quintic"


def generate_points_and_save_using_defaults(free_coefficient,number_points,force_generate=False,seed_set=0):
   coefficients=coefficientsTQ
   # coefficients=np.array([1, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, \
   # 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, free_coefficient, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 1])
   monomials=monomialsTQ
   kmoduli=kmoduliTQ
   ambient=ambientTQ
   # monomials = 5*np.eye(5, dtype=np.int64)
   # coefficients = np.ones(5)
   
   # kmoduli = np.ones(1)
   # ambient = np.array([4])
   pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
   pg._set_seed(seed_set)
   dirname = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   print("dirname: " + dirname)
   #test if the directory exists, if not, create it
   if force_generate or (not os.path.exists(dirname)):
      print("Generating: forced? " + str(force_generate))
      kappa = pg.prepare_dataset(number_points, dirname)
      pg.prepare_basis(dirname, kappa=kappa)
   elif os.path.exists(dirname):
      try:
         print("loading prexisting dataset")
         data = np.load(os.path.join(dirname, 'dataset.npz'))
         if (len(data['X_train'])+len(data['X_val']))!=number_points:
            print("wrong length - generating anyway")
            kappa = pg.prepare_dataset(number_points, dirname)
            pg.prepare_basis(dirname, kappa=kappa)
      except:
         print("error loading - generating anyway")
         kappa = pg.prepare_dataset(number_points, dirname)
         pg.prepare_basis(dirname, kappa=kappa)
   

def getcallbacksandmetrics(data):
   #rcb = RicciCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
   scb = SigmaCallback((data['X_val'], data['y_val']))
   volkcb = VolkCallback((data['X_val'], data['y_val']))
   kcb = KaehlerCallback((data['X_val'], data['y_val']))
   tcb = TransitionCallback((data['X_val'], data['y_val']))
   #cb_list = [rcb, scb, kcb, tcb, volkcb]
   cb_list = [ scb, kcb, tcb, volkcb]
   cmetrics = [TotalLoss(), SigmaLoss(), KaehlerLoss(), TransitionLoss(), VolkLoss()]#, RicciLoss()]
   return cb_list, cmetrics

def train_and_save_nn(free_coefficient,nlayer=3,nHidden=128,nEpochs=50,stddev=0.1,bSizes=[192,50000],lRate=0.001,use_zero_network=False):
   dirname = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   name = 'phimodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(bSizes[1]) + 's' + str(nlayer) + 'x' +str(nHidden)
   print('dirname: ' + dirname)
   print('name: ' + name)
   
   data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True))

   cb_list, cmetrics = getcallbacksandmetrics(data)

   act = 'gelu'
   alpha = [1., 1., 30., 1., 2.] # 1 AND 3??
   ambient=tf.cast(tf.math.abs(BASIS['AMBIENT']),tf.int32)

   #nfirstlayer=tf.reduce_prod(2*(np.array(ambient)+1)).numpy().item()
   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   #nn_phi = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   nn_phi=make_nn(10,1,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_phi_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   # print(nn_phi_zero(tf.cast(data['X_val'][0:2],tf.float32)))
   nn_phi_zero=make_nn(10,1,nlayer,nHidden,act,use_zero_network=True)
   phimodel = PhiFSModel(nn_phi, BASIS, alpha=alpha)
   phimodelzero = PhiFSModel(nn_phi_zero, BASIS, alpha=alpha)

   #Note, currently running legacy due to ongoing tf issue with M1/M2. 
   #Use the commented line instead if not on an M1/M2 machine
   opt = tfk.optimizers.Adam(learning_rate=lRate)
   #opt = tfk.optimizers.legacy.Adam(learning_rate=lRate)
   # compile so we can test on validation set before training
   phimodel.compile(custom_metrics=cmetrics)
   phimodelzero.compile(custom_metrics=cmetrics)

   ## compare validation loss before training for zero network and nonzero network
   datacasted=[tf.cast(data['X_val'],tf.float32),tf.cast(data['y_val'],tf.float32)]
   #need to re-enable learning, in case there's been a problem:
   phimodel.learn_transition = False
   phimodelzero.learn_transition = False
   phimodel.learn_volk = True
   phimodelzero.learn_volk = True
   #phimodel.learn_ricci_val= True
   #phimodelzero.learn_ricci_val= True
   valzero=phimodelzero.test_step(datacasted)
   valraw=phimodel.test_step(datacasted)
   # phimodel.learn_ricci_val=False 
   # phimodelzero.learn_ricci_val=False 
   valzero = {key: value.numpy() for key, value in valzero.items()}
   valraw = {key: value.numpy() for key, value in valraw.items()}

   phimodel, training_history = train_model(phimodel, data, optimizer=opt, epochs=nEpochs, batch_sizes=bSizes, 
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)
   return phimodel,training_history
   print("finished training\n")
   phimodel.model.save(os.path.join(dirname, name))
   np.savez_compressed(os.path.join(dirname, 'trainingHistory-' + name),training_history)
   #now print the initial losses and final losses for each metric
   # first_metrics = {key: value[0] for key, value in training_history.items()}
   # lastometrics = {key: value[-1] for key, value in training_history.items()}
   phimodel.learn_transition = True
   phimodel.learn_volk = True
   #phimodel.learn_ricci_val= True
   valfinal=phimodel.test_step(datacasted)
   valfinal = {key: value.numpy() for key, value in valfinal.items()}
   #phimodel.learn_ricci_val=False 
   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for raw network: ")
   print(valraw)
   print("validation loss for final network: ")
   print(valfinal)
   print("ratio of final to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valfinal.items()}))
   print("ratio of final to raw: " + str({key + " ratio": value/(valraw[key]+1e-8) for key, value in valfinal.items()}))

   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(phimodel,tf.cast(data["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   #IMPLEMENT THE FOLLOWING
   #meanfailuretosolveequation,_,_=measure_laplacian_failure(phimodel,data)
   print("\n\n")
   return phimodel,training_history

def load_nn_phimodel(free_coefficient,nlayer=3,nHidden=128,nEpochs=50,bSizes=[192,50000],stddev=0.1,lRate=0.001,set_weights_to_zero=False):
   dirname = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   name = 'phimodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(bSizes[1]) + 's' + str(nlayer) + 'x' +str(nHidden)
   print(dirname)
   print(name)
   
   data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True))

   cb_list, cmetrics = getcallbacksandmetrics(data)


   act = 'gelu'

   alpha = [1., 1., 30., 1., 2.] # 1 AND 3??
 


   ambient=tf.cast(BASIS['AMBIENT'],tf.int32)

   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   nn_phi = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   nn_phi_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)


#    nn_phi = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
#    nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
   phimodel = PhiFSModel(nn_phi, BASIS, alpha=alpha)
   phimodelzero = PhiFSModel(nn_phi_zero, BASIS, alpha=alpha)

   if set_weights_to_zero:
      training_history=0
   else:
      phimodel.model=tf.keras.models.load_model(os.path.join(dirname,name))
      training_history=np.load(os.path.join(dirname, 'trainingHistory-' + name +'.npz'),allow_pickle=True)['arr_0'].item()

   phimodel.compile(custom_metrics=cmetrics)
   phimodelzero.compile(custom_metrics=cmetrics)

   # compare validation loss before training for zero network and nonzero network
   datacasted=[tf.cast(data['X_val'],tf.float32),tf.cast(data['y_val'],tf.float32)]
   #need to re-enable learning, in case there's been a problem:
   phimodel.learn_transition = True
   phimodelzero.learn_transition = True
   phimodel.learn_volk = True
   phimodelzero.learn_volk = True
   #phimodel.learn_ricci_val= True
   #phimodelzero.learn_ricci_val= True
   valzero=phimodelzero.evaluate(datacasted[0],datacasted[1],return_dict=True)
   valtrained=phimodel.evaluate(datacasted[0],datacasted[1],return_dict=True)
   #metricsnames=phimodel.metrics_names
   # phimodel.learn_ricci_val=False 
   # phimodelzero.learn_ricci_val=False 
   #valzero = {metricsnames[i]: valzero[i] for i in range(len(valzero))}
   #valtrained= {metricsnames[i]: valtrained[i] for i in range(len(valtrained))}

   #valzero = {key: value.numpy() for key, value in valzero.items()}
   #valtrained = {key: value.numpy() for key, value in valtrained.items()}

   #valtrained = {key: value.numpy() for key, value in valtrained.items()}

   phimodel.learn_transition = True
   phimodel.learn_volk = True

   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for final network: ")
   print(valtrained)
   print("ratio of trained to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valtrained.items()}))
   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(phimodel,tf.cast(data["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   print("\n\n")
   #IMPLEMENT THE FOLLOWING
   #meanfailuretosolveequation,_,_=measure_laplacian_failure(phimodel,data)
   #print("\n\n")
   #print("mean of difference/mean of absolute value of source, weighted by sqrt(g): " + str(meanfailuretosolveequation))
   return phimodel,training_history

In [ ]:

def generate_points_and_save_using_defaultsAlpha(free_coefficient,phimodel,euler_char,force_generate=False,seed_set=0):
   coefficients=coefficientsTQ
   # coefficients=np.array([1, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, \
   # 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, free_coefficient, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 1])
   monomials=monomialsTQ
   kmoduli=kmoduliTQ
   ambient=ambientTQ
   # monomials = 5*np.eye(5, dtype=np.int64)
   # coefficients = np.ones(5)
   
   # kmoduli = np.ones(1)
   # ambient = np.array([4])
   pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
   pg._set_seed(seed_set)


   dirnameForMetric = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   dirnameAlpha= 'dataAlphaP/'+nameofmanifold+ "Alpha_pg_with_" + str(free_coefficient) 
   #dirnameAlpha = 'dataAlphaP/tetraquadricAlpha_pg_with_'+str(free_coefficient)+'forLB_'+lbstring
   #dirnameForMetric = 'dataAlphaP/tetraquadric_pg_with_'+str(free_coefficient)
   print("dirname for alpha: " + dirnameForMetric)
   print("dirname for alpha: " + dirnameAlpha)

   BASIS = prepare_tf_basis(np.load(os.path.join(dirnameForMetric, 'basis.pickle'), allow_pickle=True))
   
   data=np.load(os.path.join(dirnameForMetric, 'dataset.npz'))

   if force_generate or (not os.path.exists(dirnameAlpha)):
      print("Generating: forced? " + str(force_generate))
      kappaAlpha = prepare_dataset_Alpha(pg,data,dirnameAlpha,phimodel,euler_char,BASIS,normalize_to_vol_j=True);
   elif os.path.exists(dirnameAlpha):
      try:
         print("loading prexisting dataset")
         data = np.load(os.path.join(dirnameAlpha, 'dataset.npz'))
      except:
         print("problem loading data - generating anyway")
         kappaAlpha = prepare_dataset_Alpha(pg,data, dirnameAlpha,phimodel,euler_char,BASIS,normalize_to_vol_j=True);
      
   

def getcallbacksandmetricsAlpha(dataalpha):
   dataalpha_val_dict=dict(list(dict(dataalpha).items())[len(dict(dataalpha))//2:])
   tcb = TransitionCallback((dataalpha['X_val'], dataalpha['y_val']))
   lplcb = LaplacianCallback(dataalpha_val_dict)
   # lplcb = LaplacianCallback(data_val)
   cb_list = [lplcb,tcb]
   cmetrics = [TotalLoss(), LaplacianLoss(), TransitionLoss()]
   return cb_list, cmetrics

   
def train_and_save_nn_Alpha(free_coefficient,phimodel,euler_char,alphaprime,nlayer=3,nHidden=128,nEpochs=30,bSizes=[192,50000],stddev=0.1,lRate=0.001,alpha=[1,1],load_network=False,use_zero_network=False):
   
   dirnameForMetric = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   dirnameAlpha= 'dataAlphaP/'+nameofmanifold+ "Alpha_pg_with_" + str(free_coefficient) 

   #data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirnameForMetric, 'basis.pickle'), allow_pickle=True))


   dataalpha = np.load(os.path.join(dirnameAlpha, 'dataset.npz'))
   dataalpha_train=tf.data.Dataset.from_tensor_slices(dict(list(dict(dataalpha).items())[:len(dict(dataalpha))//2]))
   dataalpha_val_dict=dict(list(dict(dataalpha).items())[len(dict(dataalpha))//2:])
   dataalpha_val=tf.data.Dataset.from_tensor_slices(dataalpha_val_dict)
   # batch_sizes=[64,10000]
   dataalpha_train=dataalpha_train.shuffle(buffer_size=1024).batch(bSizes[0])

   cb_list, cmetrics = getcallbacksandmetricsAlpha(dataalpha)

   #nlayer = 3
   #nHidden = 128
   act = 'gelu'
   #nEpochs = 30
   #bSizes = [192, 150000]
   #alpha = [1., 1.] # 1 AND 3??
   nfold = 3
   n_in = 2*8
   n_out = 1
   #lRate = 0.001
   name = 'alphamodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(nlayer) + 'x' +str(nHidden)
   print("name: " + name)

   ambient=tf.cast(BASIS['AMBIENT'],tf.int32)

   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   #nfirstlayer=tf.reduce_sum(((np.array(ambient)+1)**2)).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=stddev)
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   activ=tf.square
   #activ=tfk.activations.gelu
   #nn_alpha = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #copie from phi above
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)

   #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=0.2)
   #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)#note we don't need a last bias (flat direction)
   #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network,kernel_initializer=initializer)#note we don't need a last bias (flat direction)
   #nn_alpha_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)#note we don't need a last bias (flat direction)
   
   alphamodel= AlphaPrimeModel(nn_alpha,BASIS, phimodel,alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])
   if load_network:
      print("loading network")
      alphamodel.model=tf.keras.models.load_model(os.path.join(dirnameAlpha,name))
      print("network loaded")

   alphamodelzero= AlphaPrimeModel(nn_alpha_zero,BASIS,phimodel, alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])

   #Note, currently running legacy due to ongoing tf issue with M1/M2. 
   #Use the commented line instead if not on an M1/M2 machine
   opt = tfk.optimizers.Adam(learning_rate=lRate)
   #opt = tfk.optimizers.legacy.Adam(learning_rate=lRate)
   # compile so we can test on validation set before training
   alphamodel.compile(custom_metrics=cmetrics)
   alphamodelzero.compile(custom_metrics=cmetrics)
   
   #datacasted=[tf.cast(data['X_val'],tf.float32),tf.cast(data['y_val'],tf.float32)]
   valzero=alphamodelzero.test_step(dataalpha_val_dict)
   valraw=alphamodel.test_step(dataalpha_val_dict)
   valzero = {key: value.numpy() for key, value in valzero.items()}
   valraw = {key: value.numpy() for key, value in valraw.items()}
   
   training_historyAlpha={'transition_loss': [10**(-8)],'laplacian_loss': [1000000000000000]}
   i=0
   newLR=lRate
   #while (training_historyAlpha['transition_loss'][-1]<10**(-5)) or (training_historyAlpha['laplacian_loss'][-1]>1.):
   # continue looping if >10 or is nan
   while i==0:#(training_historyAlpha['laplacian_loss'][-1]>10000000000000.) or (np.isnan( training_historyAlpha['laplacian_loss'][-1])):
      print("trying iteration of training "+str(i))
      if i >0:

         print('trying again laplacian_loss too big')
         #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=0.2)
         #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network,kernel_initializer=initializer)#note we don't need a last bias (flat direction)
         #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         #nn_alpha = BiholoModelFuncGENERALforAlphainv2(shapeofnetwork,BASIS,activation=tfk.activations.gelu,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         #nn_alpha = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=False)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)#note we don't need a last bias (flat direction)
         if newLR>0.0002:
             newLR=newLR/2
             print("new LR " + str(newLR))
         opt = tfk.optimizers.legacy.Adam(learning_rate=newLR)
         alphamodel= AlphaPrimeModel(nn_alpha,BASIS, phimodel,alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])
         cb_list, cmetrics = getcallbacksandmetricsAlpha(dataalpha)
         alphamodel.compile(custom_metrics=cmetrics)
      alphamodel, training_historyAlpha= train_modelalpha(alphamodel, dataalpha_train, optimizer=opt, epochs=nEpochs, batch_sizes=bSizes, 
                                        verbose=1, custom_metrics=cmetrics, callbacks=cb_list)
      i+=1
   print("finished training\n")
   alphamodel.model.save(os.path.join(dirnameAlpha, name))
   np.savez_compressed(os.path.join(dirnameAlpha, 'trainingHistory-' + name),training_historyAlpha)
   valfinal =alphamodel.test_step(dataalpha_val_dict)
   valfinal = {key: value.numpy() for key, value in valfinal.items()}
   #return training_historyAlpha
   #now print the initial losses and final losses for each metric, by taking the first element of each key in the dictionary
   #first_metrics = {key: value[0] for key, value in training_historyAlpha.items()}
   #last_metrics = {key: value[-1] for key, value in training_historyAlpha.items()}

   #print("initial losses")
   #print(first_metrics)
   #print("final losses")
   #print(last_metrics)


   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for raw network: ")
   print(valraw)
   print("validation loss for final network: ")
   print(valfinal)
   print("ratio of final to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valfinal.items()}))
   print("ratio of final to raw: " + str({key + " ratio": value/(valraw[key]+1e-8) for key, value in valfinal.items()}))


   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(alphamodel,tf.cast(dataalpha["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   meanfailuretosolveequation,_,_=HYM_measure_val(alphamodel,dataalpha)
   print("mean of difference/mean of absolute value of source, weighted by sqrt(g): " + str(meanfailuretosolveequation))
   print("\n\n")
   tf.keras.backend.clear_session()
   return alphamodel,training_historyAlpha

def load_nn_Alpha(free_coefficient,phimodel,euler_char,alphaprime,nlayer=3,nHidden=128,nEpochs=30,bSizes=[192,50000],stddev=0.1,lRate=0.001,alpha=[1,1],set_weights_to_zero=False):
   dirnameForMetric = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   dirnameAlpha= 'dataAlphaP/'+nameofmanifold+ "Alpha_pg_with_" + str(free_coefficient) 

   name = 'alphamodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(nlayer) + 'x' +str(nHidden)
   print("name: " + name)

   #data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirnameForMetric, 'basis.pickle'), allow_pickle=True))


   dataalpha = np.load(os.path.join(dirnameAlpha, 'dataset.npz'))
   dataalpha_train=tf.data.Dataset.from_tensor_slices(dict(list(dict(dataalpha).items())[:len(dict(dataalpha))//2]))
   dataalpha_val_dict=dict(list(dict(dataalpha).items())[len(dict(dataalpha))//2:])
   dataalpha_val=tf.data.Dataset.from_tensor_slices(dataalpha_val_dict)
   # batch_sizes=[64,10000]
   dataalpha_train=dataalpha_train.shuffle(buffer_size=1024).batch(bSizes[0])

   cb_list, cmetrics = getcallbacksandmetricsAlpha(dataalpha)

   #nlayer = 3
   #nHidden = 128
   act = 'gelu'
   #nEpochs = 30
   #bSizes = [192, 150000]
   alpha = [1., 1.] # 1 AND 3??
   nfold = 3
   n_in = 2*8
   n_out = 1
   #lRate = 0.001
   ambient=tf.cast(BASIS['AMBIENT'],tf.int32)

   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=stddev)
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   activ=tf.square
   nn_alpha = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,stddev=stddev,use_zero_network=set_weights_to_zero)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   nn_alpha_zero = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #copie from phi above
   #nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
   
   #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)#note we don't need a last bias (flat direction)
   #nn_alpha_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)#note we don't need a last bias (flat direction)
   alphamodel= AlphaPrimeModel(nn_alpha,BASIS, phimodel,alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])
   alphamodelzero= AlphaPrimeModel(nn_alpha_zero,BASIS,phimodel, alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])

   alphamodel(dataalpha['X_train'][0:1])
   alphamodelzero(dataalpha['X_train'][0:1])

   if set_weights_to_zero:
      training_historyAlpha=0
   else:
      alphamodel.model=tf.keras.models.load_model(os.path.join(dirnameAlpha,name))
      training_historyAlpha=np.load(os.path.join(dirnameAlpha, 'trainingHistory-' + name +'.npz'),allow_pickle=True)['arr_0'].item()

   alphamodel.compile(custom_metrics=cmetrics)
   alphamodelzero.compile(custom_metrics=cmetrics)
   
   valzero=alphamodelzero.evaluate(dataalpha_val_dict,return_dict=True)
   valtrained=alphamodel.evaluate(dataalpha_val_dict,return_dict=True)
   #valzero = {key: value.numpy() for key, value in valzero.items()}
   #valtrained= {key: value.numpy() for key, value in valtrained.items()}


   #metricsnames=alphamodel.metrics_names

   #valtrained= {metricsnames[i]: valtrained[i] for i in range(len(valtrained))}
   ##valzero = {metricsnames[i]: valzero[i] for i in range(len(valzero))}


   

   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for trained network: ")
   print(valtrained)
   print("ratio of trained to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valtrained.items()}))


   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(alphamodel,tf.cast(dataalpha["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   meanfailuretosolveequation,_,_=HYM_measure_val(alphamodel,dataalpha)
   print("mean of difference/mean of absolute value of source, weighted by sqrt(g): " + str(meanfailuretosolveequation))
   print("\n\n")
   return alphamodel,training_historyAlpha



Now generate example points with a point generator

Geneate the point cloud for our NN training - note that this will take a few mins


Note that "free_coefficient" is just a label for this particular quintic - for the TQ it was psi. Here, it just lets you have different runs not overwrite each other.


In [5]:
monomialsTQ = 5*np.eye(5, dtype=np.int64)
coefficientsTQ = np.ones(5)
kmoduliTQ = np.ones(1)
ambientTQ = np.array([4])
nameofmanifold="Quintic"

nPoints=500000

free_coefficient = 1.9#float(sys.argv[1])
free_coefficient=2.342351
free_coefficient=2.342343234
free_coefficient=0.00000000000001
#free_coefficient=1.# when the coefficient is 1, ensure that it's 1., not 1 for the sake of the filename
#nEpochsPhi=100
nEpochsPhi=10

depthPhi=4
widthPhi=64#128 4 in the 1.0s


train_phi=False
generate_points_and_save_using_defaults(free_coefficient,nPoints)


dirname: dataAlphaP/Quintic_pg_with_1e-14
loading prexisting dataset


## Training the NN

Now we can start preperation for training the NN

Begin by loading in the required data

In [ ]:
# phimodel2,training_history=load_nn_phimodel(free_coefficient,depthPhi,widthPhi,nEpochsPhi,[64,50000],set_weights_to_zero=False)


dataAlphaP/Quintic_pg_with_1e-14
phimodel_for_10_64_50000s4x64
network shape: [25, 64, 64, 64, 64, 1]
nns made
compiling
1563/1563 [==============================] - 6s 2ms/step - loss: 0.5476 - sigma_loss: 0.0022 - kaehler_loss: 0.0000e+00 - transition_loss: 6.9553e-10 - ricci_loss: 0.0000e+00 - volk_loss: 0.2727
zero network validation loss: 
{'loss': 0.5725029110908508, 'sigma_loss': 0.5725029110908508, 'kaehler_loss': 0.0, 'transition_loss': 0.0, 'ricci_loss': 0.0, 'volk_loss': 1.173486197103557e-08}
validation loss for final network: 
{'loss': 0.5150485634803772, 'sigma_loss': 0.0020871683955192566, 'kaehler_loss': 0.0, 'transition_loss': 6.273492281216875e-10, 'ricci_loss': 0.0, 'volk_loss': 0.25648069381713867}
ratio of trained to zero: {'loss ratio': 0.8996435555280665, 'sigma_loss ratio': 0.003645690386247029, 'kaehler_loss ratio': 0.0, 'transition_loss ratio': 0.06273492281216875, 'ricci_loss ratio': 0.0, 'volk_loss ratio': 11800428.921928806}
average transition discrepancy i

In [6]:

if train_phi:
    phimodel1,training_history=train_and_save_nn(free_coefficient,depthPhi,widthPhi,nEpochsPhi,stddev=0.05,bSizes=[64,50000],lRate=0.1) 
else:
    phimodel1,training_history=load_nn_phimodel(free_coefficient,depthPhi,widthPhi,nEpochsPhi,[64,50000],set_weights_to_zero=False)


dataAlphaP/Quintic_pg_with_1e-14
phimodel_for_10_64_50000s4x64
network shape: [25, 64, 64, 64, 64, 1]
nns made
compiling
1563/1563 [==============================] - 6s 2ms/step - loss: 0.5476 - sigma_loss: 0.0022 - kaehler_loss: 0.0000e+00 - transition_loss: 6.9553e-10 - ricci_loss: 0.0000e+00 - volk_loss: 0.2727
zero network validation loss: 
{'loss': 0.5725029110908508, 'sigma_loss': 0.5725029110908508, 'kaehler_loss': 0.0, 'transition_loss': 0.0, 'ricci_loss': 0.0, 'volk_loss': 1.173486197103557e-08}
validation loss for final network: 
{'loss': 0.5150485634803772, 'sigma_loss': 0.0020871683955192566, 'kaehler_loss': 0.0, 'transition_loss': 6.273492281216875e-10, 'ricci_loss': 0.0, 'volk_loss': 0.25648069381713867}
ratio of trained to zero: {'loss ratio': 0.8996435555280665, 'sigma_loss ratio': 0.003645690386247029, 'kaehler_loss ratio': 0.0, 'transition_loss ratio': 0.06273492281216875, 'ricci_loss ratio': 0.0, 'volk_loss ratio': 11800428.921928806}
average transition discrepancy i

In [ ]:
# import time
# data=np.load('dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) + '/dataset.npz')
# X_total=tf.cast(data['X_val'],tf.float32)[0:2048]
# cy_weights=tf.cast(data['y_val'][:,0],tf.float32)
# for batch_size in [256,384,512]:
#     #batch_size=128
#     source_computing_class= Q_compiled_function(phimodel1,X_total[0:batch_size],batch_size)    
#     start_time=time.time()
#     Q_values,euler_all_with_sqrtg = compute_batched_func(source_computing_class.compute_Q,X_total[:batch_size*10], batch_size,cy_weights)
#     print("time for batch size "+str(batch_size)+": "+str((time.time()-start_time)/(batch_size*10)))

compiling
compiled
starting christoffel tape
christoffel tape1
christoffel tape2
del christoffel tape
starting tapeR
only runs during compilation
first gradienttape
second gradienttape
tapes deleted
computing batched with batch size 256 and total length 2048


/home/f/fraser-talientec/cymetricTESTING217/lib/python3.8/site-packages/tensorflow/python/ops/gen_math_ops.py:6577: ComplexWarning: Casting complex values to real discards the imaginary part
  _result = pywrap_tfe.TFE_Py_FastPathExecute(


in 256 euler: -75588.75 vol 288.73486
in 512 euler: -71299.516 vol 287.3044
in 768 euler: -72095.62 vol 284.4358
in 1024 euler: -67482.48 vol 282.04205
in 1280 euler: -69609.33 vol 282.03625
in 1536 euler: -67711.4 vol 279.9933
in 1792 euler: -68179.35 vol 281.1218
in 2048 euler: -68394.7 vol 282.11652
time for batch size 256: 0.043286034185439345
compiling
compiled
starting christoffel tape
christoffel tape1
christoffel tape2
del christoffel tape
starting tapeR
only runs during compilation
first gradienttape
second gradienttape
tapes deleted
computing batched with batch size 384 and total length 2048
in 384 euler: -75560.4 vol 293.43234
in 768 euler: -72095.62 vol 284.4358
in 1152 euler: -68053.24 vol 281.74615
in 1536 euler: -67711.4 vol 279.9933
in 1920 euler: -68250.05 vol 281.4318
in 2304 euler: -65747.72 vol 281.90646
time for batch size 384: 0.042992850703497724
compiling
compiled
starting christoffel tape
christoffel tape1
christoffel tape2
del christoffel tape
starting tapeR
o

In [ ]:
import time
data=np.load('dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) + '/dataset.npz')
X_total=tf.cast(data['X_val'],tf.float32)[0:2048]
cy_weights=tf.cast(data['y_val'][:,0],tf.float32)
for batch_size in [256,384,512]:
    #batch_size=128
    source_computing_class= Q_compiled_function(phimodel1,X_total[0:batch_size],batch_size)    
    start_time=time.time()
    Q_values,euler_all_with_sqrtg = compute_batched_func(source_computing_class.compute_Q,X_total[:batch_size*10], batch_size,cy_weights)
    print("time for batch size "+str(batch_size)+": "+str((time.time()-start_time)/(batch_size*10)))

compiling
compiled
starting christoffel tape
christoffel tape1
christoffel tape2
del christoffel tape
starting tapeR
only runs during compilation
first gradienttape
second gradienttape
tapes deleted
computing batched with batch size 64 and total length 640


/home/f/fraser-talientec/cymetricTESTING217/lib/python3.8/site-packages/tensorflow/python/ops/gen_math_ops.py:6577: ComplexWarning: Casting complex values to real discards the imaginary part
  _result = pywrap_tfe.TFE_Py_FastPathExecute(


in 64 euler: -28083.748 vol 265.53772
in 128 euler: -54543.06 vol 279.85168
in 192 euler: -65535.902 vol 282.9739
in 256 euler: -75588.75 vol 288.73486
in 320 euler: -72049.4 vol 289.6844
in 384 euler: -75560.4 vol 293.43234
in 448 euler: -72986.875 vol 289.02023
in 512 euler: -71299.516 vol 287.3044
in 576 euler: -73450.125 vol 288.3213
in 640 euler: -75365.29 vol 287.72202
time for batch size 64: 0.06081747338175773
compiling
compiled
starting christoffel tape
christoffel tape1
christoffel tape2
del christoffel tape
starting tapeR
only runs during compilation
first gradienttape
second gradienttape
tapes deleted
computing batched with batch size 128 and total length 1280
in 128 euler: -54543.043 vol 279.85168
in 256 euler: -75588.74 vol 288.73486
in 384 euler: -75560.39 vol 293.43234
in 512 euler: -71299.516 vol 287.3044
in 640 euler: -75365.29 vol 287.72202
in 768 euler: -72095.62 vol 284.4358
in 896 euler: -68884.77 vol 282.46387
in 1024 euler: -67482.48 vol 282.04205
in 1152 euler:

: 

: 

: 

In [ ]:
# #compute ricci scalar
# from cymetric.models.measures import ricci_scalar_fn
# R_scalar_cymetric_train=ricci_scalar_fn(phimodel1,tf.cast(X_total[0:100],np.float32))

In [ ]:
# R_scalar_cymetric_train[0]
# #round to 2dp, and print in standard format
# print(np.array2string(np.round(R_scalar_cymetric_train[0], 2), formatter={'float_kind':lambda x: f"{x:.2f}"}))

[-26.42 -15.88 8.34 37.44 -21.00 58.78 -37.50 -12.50 55.89 -14.08 14.36
 -31.41 14.55 3.01 -17.73 -37.55 -36.73 12.50 -32.13 -26.40 -2.15 -34.69
 -26.19 -34.96 22.06 4.58 -1.72 5.24 64.00 48.70 -11.64 6.86 -25.40 -8.96
 24.06 -31.53 21.98 64.09 -34.84 13.21 -10.32 52.61 13.37 6.64 4.61 -18.74
 -14.62 -1.76 5.37 -34.79 -11.98 -35.05 -2.36 -10.76 -34.16 -33.27 -17.06
 11.13 -29.20 22.80 10.76 53.24 -15.38 -1.14 18.03 19.18 23.92 -34.45
 -31.54 -32.06 -36.48 22.89 38.67 -4.93 16.31 1.46 13.80 -36.13 49.92
 -35.97 50.06 5.98 2.76 -15.94 -15.63 -32.80 -11.93 -31.42 -15.20 -34.02
 -3.66 40.01 4.99 16.00 -3.19 -33.77 -5.83 -3.56 14.31 14.54]


In [7]:
generate_points_and_save_using_defaultsAlpha(free_coefficient,phimodel1,-200.,force_generate=False,seed_set=0,batch_size=128)


dirname for alpha: dataAlphaP/Quintic_pg_with_1e-14
dirname for alpha: dataAlphaP/QuinticAlpha_pg_with_1e-14
loading prexisting dataset
problem loading data - generating anyway
kappa over 6 
tf.Tensor(0.01761915, shape=(), dtype=float32)
compiling
compiled
starting christoffel tape
christoffel tape1
christoffel tape2
del christoffel tape
starting tapeR
only runs during compilation
first gradienttape
second gradienttape
tapes deleted
computing batched with batch size 128 and total length 500000


/home/f/fraser-talientec/cymetricTESTING217/lib/python3.8/site-packages/tensorflow/python/ops/gen_math_ops.py:6577: ComplexWarning: Casting complex values to real discards the imaginary part
  _result = pywrap_tfe.TFE_Py_FastPathExecute(


in 128 euler: -192.93784 vol 0.8479486
in 256 euler: -199.59787 vol 0.85239446
in 384 euler: -185.89746 vol 0.8357242
in 512 euler: -174.5135 vol 0.8177432
in 640 euler: -170.65315 vol 0.81246585
in 768 euler: -183.50537 vol 0.82104367
in 896 euler: -192.5739 vol 0.83224946
in 1024 euler: -190.34424 vol 0.8311457
in 1152 euler: -184.94272 vol 0.82402825
in 1280 euler: -195.87485 vol 0.8286231
in 1408 euler: -195.72719 vol 0.8272517
in 1536 euler: -196.35579 vol 0.8314883
in 1664 euler: -195.30574 vol 0.83052385
in 1792 euler: -193.61978 vol 0.8240824
in 1920 euler: -191.31787 vol 0.8254054
in 2048 euler: -191.44888 vol 0.8267883
in 2176 euler: -190.92386 vol 0.8275673
in 2304 euler: -188.39027 vol 0.8252374
in 2432 euler: -191.95251 vol 0.82986975
in 2560 euler: -191.71683 vol 0.8301508
in 2688 euler: -190.84465 vol 0.827933
in 2816 euler: -189.98195 vol 0.8302399
in 2944 euler: -190.3781 vol 0.8277679
in 3072 euler: -188.16199 vol 0.82433146
in 3200 euler: -188.33644 vol 0.82623285
in

In [ ]:
!pwd

/home/f/fraser-talientec


In [ ]:
alphaprime=1
euler_char=-200
depthAlpha=4
widthAlpha=64
nEpochsAlpha=30


train_alpha=True
if train_alpha:
    AlphaModel1,training_historyAlpha=train_and_save_nn_Alpha(free_coefficient,phimodel1,euler_char,alphaprime,depthAlpha,widthAlpha,nEpochsAlpha,bSizes=[64,50000],stddev=0.05,lRate=0.1,use_zero_network=False,alpha=[1.,1.],load_network=False)
else:
    AlphaModel1,training_historyAlpha=load_nn_Alpha(free_coefficient,phimodel1,euler_char,alphaprime,depthPhi,widthPhi,nEpochsPhi,[64,50000],set_weights_to_zero=True)

FileNotFoundError: [Errno 2] No such file or directory: 'dataAlphaP/QuinticAlpha_pg_with_1e-14/dataset.npz'

In [ ]:
data=np.load('dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) + '/dataset.npz')
# AlphaModel1.model(data['X_train'][0:1])

In [ ]:
#reimport custom_networks
importlib.reload(sys.modules['custom_networks'])
from custom_networks import *
#reimport laplacian_funcs
importlib.reload(sys.modules['laplacian_funcs'])
from laplacian_funcs import *

In [ ]:
#(self,kphi,linebundleindices,ambient_env_var,n_projective,kmoduli)
n_0=0
n_p= 500000*9//10
ptdata=tf.cast(data['X_train'],tf.float32)[n_0:n_p]
#cptdata=point_vec_to_complex(ptdata)
#cptdata2=cptdata/cptdata[:,0:1]
flat_measure_weights=tf.cast(((data['y_train'][:,0]/(data['y_train'][:,1]))*(1/(6)))[n_0:n_p],tf.float32)
pullbacks=AlphaModel1.pullbacks(ptdata)#[n_0:n_p]
kphi = np.array([3])
#get_sections_cpts_func=get_degree_kphiandMmonomials_general(kphi,[0],ambientTQ,len(ambientTQ),kmoduliTQ)
# get_sections_cpts_func(cptdata)


In [ ]:

#tf.function this, but for fixed arguments 0,2 and tensor arguments 1,3,4

def compute_matrices_for_pts(rpoints,metric,flat_measure_weights,pullbacks,get_sections_cpts_func):
    get_sections_rpts_func= lambda x: get_sections_cpts_func(point_vec_to_complex(x))
    metrics = metric(rpoints)
    inversemetrics=tf.linalg.inv(metrics)
    dets = tf.linalg.det(metrics)
    #k_phi=get_sections_rpts_func(cpoints)
    djsigma=extder_j_for_sigma(rpoints,get_sections_rpts_func)
    djbarsigma=extder_jbar_for_sigma(rpoints,get_sections_rpts_func)
    dasigma = tf.einsum('xai,xCi->xCa',pullbacks,djsigma)
    dbbarsigma = tf.einsum('xbj,xCj->xCb',tf.math.conj(pullbacks),djbarsigma)
    #b is barred, a is unbarred
    dbarfbar_df = tf.einsum('xba,xCb,xDa->xCD',inversemetrics,tf.math.conj(dasigma),dasigma)
    dfbar_dbarf = tf.einsum('xba,xCa,xDb->xCD',inversemetrics,tf.math.conj(dbbarsigma),dbbarsigma)
    #factor of 2 form the laplacian!
    integrand_CD = tf.einsum('xCD,x->xCD',2*(dbarfbar_df + dfbar_dbarf),dets*tf.cast(flat_measure_weights,tf.complex64))
    integrand_mean=tf.reduce_mean(integrand_CD,axis=0)
    print("vol: " + str(tf.reduce_mean(dets*tf.cast(flat_measure_weights,tf.complex64))))
    return integrand_mean
    
get_sections_cpts_func=get_degree_kphiandMmonomials_general(kphi,kphi*0,ambientTQ,len(ambientTQ),kmoduliTQ)
#mats=compute_matrices_for_pts(kphi,ptdata,AlphaModel1,flat_measure_weights,pullbacks,get_sections_cpts_func)



In [ ]:
def compute_normalising_matrices_for_pts(rpoints,metric,flat_measure_weights,get_sections_cpts_func):
    get_sections_rpts_func= lambda x: get_sections_cpts_func(point_vec_to_complex(x))
    metrics = metric(rpoints)
    dets = tf.linalg.det(metrics)
    sectionsB    = get_sections_rpts_func(rpoints)
    sectionsConjA   = tf.math.conj(sectionsB)

    #k_phi=get_sections_rpts_func(cpoints)
    integrand_CD = tf.einsum('xA,xB,x->AB',sectionsConjA,sectionsB,dets*tf.cast(flat_measure_weights,tf.complex64))/tf.cast(tf.shape(flat_measure_weights)[-1],tf.complex64)
    print("vol: " + str(tf.reduce_mean(dets*tf.cast(flat_measure_weights,tf.complex64))))
    return integrand_CD

In [ ]:
batch_size=100
metric_using=phimodel1


#create compiled versions
@tf.function
def compute_matrices_for_pts_fixed(rpoints, flat_measure_weights, pullbacks):
    #kphi = tf.constant([1])  # Fixed argument
    return compute_matrices_for_pts(rpoints, metric_using, flat_measure_weights, pullbacks,get_sections_cpts_func)

# Get the concrete function
concrete_func = compute_matrices_for_pts_fixed.get_concrete_function(
    tf.TensorSpec(shape=(batch_size,2*metric_using.ncoords), dtype=tf.float32),  # rpoints
    tf.TensorSpec(shape=(batch_size), dtype=tf.float32),  # flat_measure_weights
    tf.TensorSpec(shape=(batch_size, metric_using.nfold,metric_using.ncoords), dtype=tf.complex64)  # pullbacks
)

@tf.function
def compute_normalising_matrices_for_pts_fixed(rpoints, flat_measure_weights):
    #kphi = tf.constant([1])  # Fixed argument
    return compute_normalising_matrices_for_pts(rpoints, metric_using, flat_measure_weights,get_sections_cpts_func)

# Get the concrete function for the normalising matrix
concrete_func_normalising = compute_normalising_matrices_for_pts_fixed.get_concrete_function(
    tf.TensorSpec(shape=(batch_size,2*metric_using.ncoords), dtype=tf.float32),  # rpoints
    tf.TensorSpec(shape=(batch_size), dtype=tf.float32),  # flat_measure_weights
)

#compute_matrices_for_pts_fixed(ptdata, flat_measure_weights, pullbacks)

In [ ]:
#test runtime!

dataset = tf.data.Dataset.from_tensor_slices((ptdata, flat_measure_weights, pullbacks))
batched_dataset = dataset.batch(batch_size)
import time
start = time.time()
for batch in batched_dataset.take(4):
    batch_rpoints, batch_flat_measure_weights, batch_pullbacks = batch
    batch_result = compute_matrices_for_pts_fixed(batch_rpoints, batch_flat_measure_weights, batch_pullbacks)
    #oresults.append(batch_result)
    print('done, timing: ' + str(time.time()-start))
    start=time.time()

print("\n\n concrete" )
import time
start = time.time()
for batch in batched_dataset.take(4):
    batch_rpoints, batch_flat_measure_weights, batch_pullbacks = batch
    batch_result = concrete_func(batch_rpoints, batch_flat_measure_weights, batch_pullbacks)
    #oresults.append(batch_result)
    print('done, timing: ' + str(time.time()-start))
    start=time.time()

In [ ]:
# Assuming the concrete_func is defined as in the previous example

# Create a dataset from your large number of points
dataset = tf.data.Dataset.from_tensor_slices((ptdata, flat_measure_weights, pullbacks))
# Adjust based on your memory constraints and performance needs


# Batch the dataset
batched_dataset = dataset.batch(batch_size)

# Process batches
results = []
for batch in batched_dataset:
    batch_rpoints, batch_flat_measure_weights, batch_pullbacks = batch
    batch_result = concrete_func(batch_rpoints, batch_flat_measure_weights, batch_pullbacks)
    results.append(batch_result)


# Combine results if needed
matrix_laplacian_requires_mean = np.array(results)

print(f"Processed {num_points} points. Shape of final result: {final_result.shape}")

results_norm = []
for batch in batched_dataset:
    batch_rpoints, batch_flat_measure_weights,_ = batch
    batch_result_norm = concrete_func_normalising(batch_rpoints, batch_flat_measure_weights)
    results_norm.append(batch_result_norm)

matrix_normalising_requires_mean = np.array(results_norm)

print(f"Processed {num_points} points. Shape of final result: {final_result_norm.shape}")

In [ ]:
import scipy
#take eigs of final_result


#log distribute over 1 to 100000
number_approx=6

numbers_to_take= np.unique((batch_size*np.round(np.logspace(np.log10(10000),np.log10(len(ptdata)),number_approx)/batch_size)).astype('int'))[0:]
indices_to_take=tf.cast(numbers_to_take/batch_size,tf.int32).numpy()
indices_to_take[0]
print(numbers_to_take)

eigvals_all=[]
for i in indices_to_take:
    matrix=tf.reduce_mean(np.array(results)[0:i],axis=0)
    matrix_norm=tf.reduce_mean(np.array(results_norm)[0:i],axis=0)
    eigvals=tf.math.real(scipy.linalg.eigvals(matrix,matrix_norm))
    eigvals_all.append(np.sort(eigvals))
    #round and print to 2dp
    #eigvals_rounded=np.round(eigvals,2)
    #print(tf.math.real(eigvals_rounded))
eigvals_all=np.array(eigvals_all)

In [ ]:
import matplotlib.pyplot as plt
#sort_eigvals_rounded=np.sort(eigvals_rounded)[0:200]
#plt.scatter(range(len(sort_eigvals_rounded)),sort_eigvals_rounded)
numbers_for_plotting=tf.repeat(tf.expand_dims(numbers_to_take,axis=-1),tf.shape(eigvals_all)[-1],axis=-1)
#join each eigval as hte number of points increase with a dotted line
#change the plot to a line plot
#change all the line colours to red and thin, with a small red x at each point
#fade the lines from blue to red corresponding to their index in the first axis of the array

#plt.plot(numbers_for_plotting,eigvals_all,marker='x',linestyle='dotted',color='red',linewidth=0.5,markersize=1)


#change the x axis to log scale, labelled at the endpoints with one point in the middle
plt.xscale('log')
middle_pt=((numbers_to_take[-1]-numbers_to_take[0])//(5000*2))*5000 + numbers_to_take[0]
middle_pt2=((numbers_to_take[-1]-numbers_to_take[0])//(5000*5))*5000 + numbers_to_take[0]
ticks = tf.cast(np.concatenate((numbers_to_take[[0,-1]],np.array([middle_pt,middle_pt2])),axis=0),tf.int32).numpy()
plt.xticks(ticks,ticks)
#label the x axis as number of points n_p
#label the y axis as eigenvalue
plt.xlabel('number of points n_p')
plt.ylabel('eigenvalue')
#title the plot as eigenvalues of the laplacian as number of points increases
plt.title('eigenvalues of the laplacian vs integration dataset size')
plt.ylim(0,300)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Create a custom colormap from blue to red
colors = [(0, 0, 1), (1, 0, 0)]  # R -> G -> B
n_bins = eigvals_all.shape[1]  # Number of color bins (1225 eigenvalues)
cmap = LinearSegmentedColormap.from_list('red_to_blue', colors, N=n_bins)

# Plot setup
fig, ax = plt.subplots(figsize=(10, 6))

# Plot each eigenvalue evolution with a different color
for i in range(n_bins):
    color = cmap(i / (n_bins - 1))
    ax.plot(numbers_for_plotting[:,i], eigvals_all[:,i],
            marker='x', linestyle='dotted', color=color, linewidth=0.5, markersize=1)

# Set x-axis to log scale
ax.set_xscale('log')

# Set x-axis ticks
middle_pt = ((numbers_to_take[-1]-numbers_to_take[0])//(5000*2))*5000 + numbers_to_take[0]
middle_pt2 = ((numbers_to_take[-1]-numbers_to_take[0])//(5000*5))*5000 + numbers_to_take[0]
ticks = np.concatenate((numbers_to_take[[0,-1]], np.array([middle_pt, middle_pt2])))
ax.set_xticks(ticks)
ax.set_xticklabels(ticks)

# Set labels and title
ax.set_xlabel('number of points n_p')
ax.set_ylabel('eigenvalue')
ax.set_title('eigenvalues of the laplacian vs integration dataset size')

# Set y-axis limit
# `ax.set_ylim(0, 300)`

plt.tight_layout()
plt.show()

In [3]:

def create_adam_optimizer_with_decay(initial_learning_rate, nEpochs, final_lr_factor=10):
    """
    Creates an Adam optimizer with exponential learning rate decay.

    Parameters:
    - initial_learning_rate: The starting learning rate
    - nEpochs: The number of epochs over which to decay the learning rate
    - final_lr_factor: The factor by which to reduce the learning rate (default: 10)

    Returns:
    - An Adam optimizer with the specified learning rate decay
    """
    decay_rate = (1/final_lr_factor) ** (1/nEpochs)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=1,
        decay_rate=decay_rate,
        staircase=False
    )

    return tf.keras.optimizers.Adam(learning_rate=lr_schedule)

def generate_points_and_save_using_defaults(free_coefficient,number_points,force_generate=False,seed_set=0):
   coefficients=coefficientsTQ
   # coefficients=np.array([1, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, \
   # 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, free_coefficient, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 1])
   monomials=monomialsTQ
   kmoduli=kmoduliTQ
   ambient=ambientTQ
   pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
   pg._set_seed(seed_set)
   #dirname = 'data/tetraquadric_pg_wit#h_'+str(free_coefficient)
   dirname = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   print("dirname: " + dirname)
   #test if the directory exists, if not, create it
   if force_generate or (not os.path.exists(dirname)):
      print("Generating: forced? " + str(force_generate))
      kappa = pg.prepare_dataset(number_points, dirname)
      pg.prepare_basis(dirname, kappa=kappa)
   elif os.path.exists(dirname):
      try:
         print("loading prexisting dataset")
         data = np.load(os.path.join(dirname, 'dataset.npz'))
         if (len(data['X_train'])+len(data['X_val']))!=number_points:
            print("wrong length - generating anyway")
            kappa = pg.prepare_dataset(number_points, dirname)
            pg.prepare_basis(dirname, kappa=kappa)
      except:
         print("error loading - generating anyway")
         kappa = pg.prepare_dataset(number_points, dirname)
         pg.prepare_basis(dirname, kappa=kappa)


def getcallbacksandmetrics(data):
   #rcb = RicciCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
   scb = SigmaCallback((data['X_val'], data['y_val']))
   volkcb = VolkCallback((data['X_val'], data['y_val']))
   kcb = KaehlerCallback((data['X_val'], data['y_val']))
   tcb = TransitionCallback((data['X_val'], data['y_val']))
   #cb_list = [rcb, scb, kcb, tcb, volkcb]
   #cb_list = [ scb, kcb, tcb, volkcb]
   #cmetrics = [TotalLoss(), SigmaLoss(), KaehlerLoss(), TransitionLoss(), VolkLoss()]#, RicciLoss()]
   cb_list = [ scb ]
   cmetrics = [TotalLoss(), SigmaLoss()]#, RicciLoss()]
   return cb_list, cmetrics

def train_and_save_nn(free_coefficient,nlayer=3,nHidden=128,nEpochs=50,stddev=0.1,bSizes=[192,50000],lRate=0.001,use_zero_network=False):
   #dirname = 'data/tetraquadric_pg_with_'+str(free_coefficient)
   dirname = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   name = 'phimodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(bSizes[1]) + 's' + str(nlayer) + 'x' +str(nHidden)
   print('dirname: ' + dirname)
   print('name: ' + name)

   data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True))

   cb_list, cmetrics = getcallbacksandmetrics(data)

   #nlayer = 3
   #nHidden = 128
   act = 'gelu'
   #nEpochs = 100
   #bSizes = [192, 150000]
   alpha = [1., 1., 30., 1., 2.] # 1 AND 3??
   #nfold = 3
   #n_in = 2*5
   #n_out = 1
   #lRate = 0.001
   ambient=tf.cast(tf.math.abs(BASIS['AMBIENT']),tf.int32)

   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   nn_phi = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   nn_phi_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
   phimodel = PhiFSModel(nn_phi, BASIS, alpha=alpha)
   phimodelzero = PhiFSModel(nn_phi_zero, BASIS, alpha=alpha)
   #print('nn_phi ' )
   #print('nn_phi ' + str(phimodel(data['X_train'][0:10])))

   #Note, currently running legacy due to ongoing tf issue with M1/M2.
   #Use the commented line instead if not on an M1/M2 machine
   #opt = tfk.optimizers.Adam(learning_rate=lRate)
   #opt = tfk.optimizers.legacy.Adam(learning_rate=lRate)
   opt='adam'
   #opt = create_adam_optimizer_with_decay(
   # initial_learning_rate=lRate,
   # nEpochs=nEpochs*(len(data['X_train'])//bSizes[0]),
   # final_lr_factor=10  # This will decay to lRate/10
   #)
   # compile so we can test on validation set before training
   phimodel.compile(custom_metrics=cmetrics)
   phimodelzero.compile(custom_metrics=cmetrics)

   ## compare validation loss before training for zero network and nonzero network
   datacasted=[tf.cast(data['X_val'],tf.float32),tf.cast(data['y_val'],tf.float32)]
   #need to re-enable learning, in case there's been a problem:
   phimodel.learn_transition = False
   phimodelzero.learn_transition = False
   phimodel.learn_volk = True
   phimodelzero.learn_volk = True
   #phimodel.learn_ricci_val= True
   #phimodelzero.learn_ricci_val= True
   print("testing")
   valzero=phimodelzero.test_step(datacasted)
   valraw=phimodel.test_step(datacasted)
   print('tested')
   # phimodel.learn_ricci_val=False
   # phimodelzero.learn_ricci_val=False
   valzero = {key: value.numpy() for key, value in valzero.items()}
   valraw = {key: value.numpy() for key, value in valraw.items()}

   #### maybe remove
   phimodel.learn_volk = False
   phimodel.learn_transition=False
   phimodel, training_history = train_model(phimodel, data, optimizer=opt, epochs=nEpochs, batch_sizes=bSizes,
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)
   print("finished training\n")
   phimodel.model.save_weights(os.path.join(dirname, name) + '.weights.h5')
   np.savez_compressed(os.path.join(dirname, 'trainingHistory-' + name),training_history)
   #now print the initial losses and final losses for each metric
   # first_metrics = {key: value[0] for key, value in training_history.items()}
   # lastometrics = {key: value[-1] for key, value in training_history.items()}
   phimodel.learn_transition = True
   phimodel.learn_volk = True
   #phimodel.learn_ricci_val= True
   valfinal=phimodel.test_step(datacasted)
   valfinal = {key: value.numpy() for key, value in valfinal.items()}
   #phimodel.learn_ricci_val=False
   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for raw network: ")
   print(valraw)
   print("validation loss for final network: ")
   print(valfinal)
   print("ratio of final to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valfinal.items()}))
   print("ratio of final to raw: " + str({key + " ratio": value/(valraw[key]+1e-8) for key, value in valfinal.items()}))

   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(phimodel,tf.cast(data["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   #IMPLEMENT THE FOLLOWING
   #meanfailuretosolveequation,_,_=measure_laplacian_failure(phimodel,data)
   print("\n\n")
   return phimodel,training_history

def load_nn_phimodel(free_coefficient,nlayer=3,nHidden=128,nEpochs=50,bSizes=[192,50000],stddev=0.1,lRate=0.001,set_weights_to_zero=False):
   dirname = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   name = 'phimodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(bSizes[1]) + 's' + str(nlayer) + 'x' +str(nHidden)
   print(dirname)
   print(name)

   data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True))

   cb_list, cmetrics = getcallbacksandmetrics(data)

   #nlayer = 3
   #nHidden = 128
   act = 'gelu'
   #nEpochs = 100
   #bSizes = [192, 150000]
   alpha = [1., 1., 30., 1., 2.] # 1 AND 3??
   nfold = 3
   n_in = 2*8
   n_out = 1
   #lRate = 0.001


   ambient=tf.cast(BASIS['AMBIENT'],tf.int32)

   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   nn_phi = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   nn_phi_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
   print("nns made")

   datacasted=[tf.cast(data['X_val'],tf.float32),tf.cast(data['y_val'],tf.float32)]

   #    nn_phi = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
   #    nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
   phimodel = PhiFSModel(nn_phi, BASIS, alpha=alpha)
   phimodelzero = PhiFSModel(nn_phi_zero, BASIS, alpha=alpha)
   #initialise weights
   phimodel(datacasted[0][0:1])
   phimodelzero(datacasted[0][0:1])


   if set_weights_to_zero:
      training_history=0
   else:
      #phimodel.model=tf.keras.layers.TFSMLayer(os.path.join(dirname,name),call_endpoint="serving_default")
      #phimodel.model=tf.keras.models.load_model(os.path.join(dirname, name) + ".keras")
      #print_sample_params(phimodel.model)
      phimodel.model.load_weights(os.path.join(dirname, name) + '.weights.h5')
      #print_sample_params(phimodel.model)
      training_history=np.load(os.path.join(dirname, 'trainingHistory-' + name +'.npz'),allow_pickle=True)['arr_0'].item()

   print("compiling")
   phimodel.compile(custom_metrics=cmetrics)
   phimodelzero.compile(custom_metrics=cmetrics)

   # compare validation loss before training for zero network and nonzero network
   #need to re-enable learning, in case there's been a problem:
   phimodel.learn_transition = True
   phimodelzero.learn_transition = True
   phimodel.learn_volk = True
   phimodelzero.learn_volk = True

   #problem - metricsnames aren't defined unless model has been trained, not just evaluated? SOlution - return_dict
   valzero=phimodelzero.evaluate(datacasted[0],datacasted[1],return_dict=True)
   valtrained=phimodel.evaluate(datacasted[0],datacasted[1],return_dict=True)


   phimodel.learn_transition = True
   phimodel.learn_volk = True

   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for final network: ")
   print(valtrained)
   print("ratio of trained to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valtrained.items()}))
   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(phimodel,tf.cast(data["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   print("\n\n")
   #IMPLEMENT THE FOLLOWING
   #meanfailuretosolveequation,_,_=measure_laplacian_failure(phimodel,data)
   #print("\n\n")
   #print("mean of difference/mean of absolute value of source, weighted by sqrt(g): " + str(meanfailuretosolveequation))
   tf.keras.backend.clear_session()
   return phimodel,training_history

In [4]:

def generate_points_and_save_using_defaultsAlpha(free_coefficient,phimodel,euler_char,force_generate=False,seed_set=0,batch_size=128):
   coefficients=coefficientsTQ
   # coefficients=np.array([1, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, \
   # 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, free_coefficient, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, \
   # 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 1])
   monomials=monomialsTQ
   kmoduli=kmoduliTQ
   ambient=ambientTQ
   # monomials = 5*np.eye(5, dtype=np.int64)
   # coefficients = np.ones(5)
   
   # kmoduli = np.ones(1)
   # ambient = np.array([4])
   pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
   pg._set_seed(seed_set)


   dirnameForMetric = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   dirnameAlpha= 'dataAlphaP/'+nameofmanifold+ "Alpha_pg_with_" + str(free_coefficient) 
   #dirnameAlpha = 'dataAlphaP/tetraquadricAlpha_pg_with_'+str(free_coefficient)+'forLB_'+lbstring
   #dirnameForMetric = 'dataAlphaP/tetraquadric_pg_with_'+str(free_coefficient)
   print("dirname for alpha: " + dirnameForMetric)
   print("dirname for alpha: " + dirnameAlpha)

   BASIS = prepare_tf_basis(np.load(os.path.join(dirnameForMetric, 'basis.pickle'), allow_pickle=True))
   
   data=np.load(os.path.join(dirnameForMetric, 'dataset.npz'))

   if force_generate or (not os.path.exists(dirnameAlpha)):
      print("Generating: forced? " + str(force_generate))
      kappaAlpha = prepare_dataset_Alpha(pg,data,dirnameAlpha,phimodel,euler_char,BASIS,normalize_to_vol_j=True,batch_size=batch_size);
   elif os.path.exists(dirnameAlpha):
      try:
         print("loading prexisting dataset")
         data = np.load(os.path.join(dirnameAlpha, 'dataset.npz'))
      except:
         print("problem loading data - generating anyway")
         kappaAlpha = prepare_dataset_Alpha(pg,data, dirnameAlpha,phimodel,euler_char,BASIS,normalize_to_vol_j=True,batch_size=batch_size);
      
   

def getcallbacksandmetricsAlpha(dataalpha):
   dataalpha_val_dict=dict(list(dict(dataalpha).items())[len(dict(dataalpha))//2:])
   tcb = TransitionCallback((dataalpha['X_val'], dataalpha['y_val']))
   lplcb = LaplacianCallback(dataalpha_val_dict)
   # lplcb = LaplacianCallback(data_val)
   cb_list = [lplcb,tcb]
   cmetrics = [TotalLoss(), LaplacianLoss(), TransitionLoss()]
   return cb_list, cmetrics

   
def train_and_save_nn_Alpha(free_coefficient,phimodel,euler_char,alphaprime,nlayer=3,nHidden=128,nEpochs=30,bSizes=[192,50000],stddev=0.1,lRate=0.001,alpha=[1,1],load_network=False,use_zero_network=False):
   
   dirnameForMetric = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   dirnameAlpha= 'dataAlphaP/'+nameofmanifold+ "Alpha_pg_with_" + str(free_coefficient) 

   #data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirnameForMetric, 'basis.pickle'), allow_pickle=True))


   dataalpha = np.load(os.path.join(dirnameAlpha, 'dataset.npz'))
   dataalpha_train=tf.data.Dataset.from_tensor_slices(dict(list(dict(dataalpha).items())[:len(dict(dataalpha))//2]))
   dataalpha_val_dict=dict(list(dict(dataalpha).items())[len(dict(dataalpha))//2:])
   dataalpha_val=tf.data.Dataset.from_tensor_slices(dataalpha_val_dict)
   # batch_sizes=[64,10000]
   dataalpha_train=dataalpha_train.shuffle(buffer_size=1024).batch(bSizes[0])

   cb_list, cmetrics = getcallbacksandmetricsAlpha(dataalpha)

   #nlayer = 3
   #nHidden = 128
   act = 'gelu'
   #nEpochs = 30
   #bSizes = [192, 150000]
   #alpha = [1., 1.] # 1 AND 3??
   nfold = 3
   n_in = 2*8
   n_out = 1
   #lRate = 0.001
   name = 'alphamodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(nlayer) + 'x' +str(nHidden)
   print("name: " + name)

   ambient=tf.cast(BASIS['AMBIENT'],tf.int32)

   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   #nfirstlayer=tf.reduce_sum(((np.array(ambient)+1)**2)).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=stddev)
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   activ=tf.square
   #activ=tfk.activations.gelu
   #nn_alpha = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #copie from phi above
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero =BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)

   #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=0.2)
   #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)#note we don't need a last bias (flat direction)
   #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network,kernel_initializer=initializer)#note we don't need a last bias (flat direction)
   #nn_alpha_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)#note we don't need a last bias (flat direction)
   
   alphamodel= AlphaPrimeModel(nn_alpha,BASIS, phimodel,alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])
   if load_network:
      print("loading network")
      alphamodel.model=tf.keras.models.load_model(os.path.join(dirnameAlpha,name))
      print("network loaded")

   alphamodelzero= AlphaPrimeModel(nn_alpha_zero,BASIS,phimodel, alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])

   #Note, currently running legacy due to ongoing tf issue with M1/M2. 
   #Use the commented line instead if not on an M1/M2 machine
   opt = tfk.optimizers.Adam(learning_rate=lRate)
   #opt = tfk.optimizers.legacy.Adam(learning_rate=lRate)
   # compile so we can test on validation set before training
   alphamodel.compile(custom_metrics=cmetrics)
   alphamodelzero.compile(custom_metrics=cmetrics)
   
   #datacasted=[tf.cast(data['X_val'],tf.float32),tf.cast(data['y_val'],tf.float32)]
   valzero=alphamodelzero.test_step(dataalpha_val_dict)
   valraw=alphamodel.test_step(dataalpha_val_dict)
   valzero = {key: value.numpy() for key, value in valzero.items()}
   valraw = {key: value.numpy() for key, value in valraw.items()}
   
   training_historyAlpha={'transition_loss': [10**(-8)],'laplacian_loss': [1000000000000000]}
   i=0
   newLR=lRate
   #while (training_historyAlpha['transition_loss'][-1]<10**(-5)) or (training_historyAlpha['laplacian_loss'][-1]>1.):
   # continue looping if >10 or is nan
   while i==0:#(training_historyAlpha['laplacian_loss'][-1]>10000000000000.) or (np.isnan( training_historyAlpha['laplacian_loss'][-1])):
      print("trying iteration of training "+str(i))
      if i >0:

         print('trying again laplacian_loss too big')
         #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=0.2)
         #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network,kernel_initializer=initializer)#note we don't need a last bias (flat direction)
         #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         #nn_alpha = BiholoModelFuncGENERALforAlphainv2(shapeofnetwork,BASIS,activation=tfk.activations.gelu,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         #nn_alpha = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,stddev=stddev,use_zero_network=use_zero_network)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=False)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
         #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)#note we don't need a last bias (flat direction)
         if newLR>0.0002:
             newLR=newLR/2
             print("new LR " + str(newLR))
         opt = tfk.optimizers.legacy.Adam(learning_rate=newLR)
         alphamodel= AlphaPrimeModel(nn_alpha,BASIS, phimodel,alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])
         cb_list, cmetrics = getcallbacksandmetricsAlpha(dataalpha)
         alphamodel.compile(custom_metrics=cmetrics)
      alphamodel, training_historyAlpha= train_modelalpha(alphamodel, dataalpha_train, optimizer=opt, epochs=nEpochs, batch_sizes=bSizes, 
                                        verbose=1, custom_metrics=cmetrics, callbacks=cb_list)
      i+=1
   print("finished training\n")
   #alphamodel.model.save(os.path.join(dirnameAlpha, name))
   phimodel.model.save_weights(os.path.join(dirname, name) + '.weights.h5')
   np.savez_compressed(os.path.join(dirnameAlpha, 'trainingHistory-' + name),training_historyAlpha)
   valfinal =alphamodel.test_step(dataalpha_val_dict)
   valfinal = {key: value.numpy() for key, value in valfinal.items()}
   #return training_historyAlpha
   #now print the initial losses and final losses for each metric, by taking the first element of each key in the dictionary
   #first_metrics = {key: value[0] for key, value in training_historyAlpha.items()}
   #last_metrics = {key: value[-1] for key, value in training_historyAlpha.items()}

   #print("initial losses")
   #print(first_metrics)
   #print("final losses")
   #print(last_metrics)


   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for raw network: ")
   print(valraw)
   print("validation loss for final network: ")
   print(valfinal)
   print("ratio of final to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valfinal.items()}))
   print("ratio of final to raw: " + str({key + " ratio": value/(valraw[key]+1e-8) for key, value in valfinal.items()}))


   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(alphamodel,tf.cast(dataalpha["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   meanfailuretosolveequation,_,_=HYM_measure_val(alphamodel,dataalpha)
   print("mean of difference/mean of absolute value of source, weighted by sqrt(g): " + str(meanfailuretosolveequation))
   print("\n\n")
   tf.keras.backend.clear_session()
   return alphamodel,training_historyAlpha

def load_nn_Alpha(free_coefficient,phimodel,euler_char,alphaprime,nlayer=3,nHidden=128,nEpochs=30,bSizes=[192,50000],stddev=0.1,lRate=0.001,alpha=[1,1],set_weights_to_zero=False):
   dirnameForMetric = 'dataAlphaP/'+nameofmanifold+ "_pg_with_" + str(free_coefficient) 
   dirnameAlpha= 'dataAlphaP/'+nameofmanifold+ "Alpha_pg_with_" + str(free_coefficient) 

   name = 'alphamodel_for_' + str(nEpochs) + '_' + str(bSizes[0]) + '_'+ str(nlayer) + 'x' +str(nHidden)
   print("name: " + name)

   #data = np.load(os.path.join(dirname, 'dataset.npz'))
   BASIS = prepare_tf_basis(np.load(os.path.join(dirnameForMetric, 'basis.pickle'), allow_pickle=True))


   dataalpha = np.load(os.path.join(dirnameAlpha, 'dataset.npz'))
   dataalpha_train=tf.data.Dataset.from_tensor_slices(dict(list(dict(dataalpha).items())[:len(dict(dataalpha))//2]))
   dataalpha_val_dict=dict(list(dict(dataalpha).items())[len(dict(dataalpha))//2:])
   dataalpha_val=tf.data.Dataset.from_tensor_slices(dataalpha_val_dict)
   # batch_sizes=[64,10000]
   dataalpha_train=dataalpha_train.shuffle(buffer_size=1024).batch(bSizes[0])

   cb_list, cmetrics = getcallbacksandmetricsAlpha(dataalpha)

   #nlayer = 3
   #nHidden = 128
   act = 'gelu'
   #nEpochs = 30
   #bSizes = [192, 150000]
   alpha = [1., 1.] # 1 AND 3??
   nfold = 3
   n_in = 2*8
   n_out = 1
   #lRate = 0.001
   ambient=tf.cast(BASIS['AMBIENT'],tf.int32)

   nfirstlayer=tf.reduce_prod((np.array(ambient)+1)**2).numpy().item()
   shapeofinternalnetwork=[nHidden]*nlayer
   shapeofnetwork=[nfirstlayer]+shapeofinternalnetwork+[1]

   print("network shape: " + str(shapeofnetwork))
   #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=stddev)
   #nn_alpha = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,stddev=stddev,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #nn_alpha_zero = BiholoModelFuncGENERAL(shapeofnetwork,BASIS,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   activ=tf.square
   nn_alpha = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,stddev=stddev,use_zero_network=set_weights_to_zero)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   nn_alpha_zero = BiholoModelFuncGENERALforHYMinv3(shapeofnetwork,BASIS,activation=activ,use_zero_network=True)#make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=use_zero_network)
   #copie from phi above
   #nn_phi_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)
   
   #nn_alpha = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)#note we don't need a last bias (flat direction)
   #nn_alpha_zero = make_nn(n_in,n_out,nlayer,nHidden,act,use_zero_network=True)#note we don't need a last bias (flat direction)
   alphamodel= AlphaPrimeModel(nn_alpha,BASIS, phimodel,alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])
   alphamodelzero= AlphaPrimeModel(nn_alpha_zero,BASIS,phimodel, alphaprime,euler_char,alpha=alpha,norm = [1. for _ in range(2)])

   alphamodel(dataalpha['X_train'][0:1])
   alphamodelzero(dataalpha['X_train'][0:1])


   if set_weights_to_zero:
      training_historyAlpha=0
   else:
      #alphamodel.model=tf.keras.models.load_model(os.path.join(dirnameAlpha,name))
      alphamodel.model.load_weights(os.path.join(dirnameAlpha, name) + '.weights.h5')
      training_historyAlpha=np.load(os.path.join(dirnameAlpha, 'trainingHistory-' + name +'.npz'),allow_pickle=True)['arr_0'].item()

   alphamodel.compile(custom_metrics=cmetrics)
   alphamodelzero.compile(custom_metrics=cmetrics)
   
   valzero=alphamodelzero.evaluate(dataalpha_val_dict,return_dict=True)
   valtrained=alphamodel.evaluate(dataalpha_val_dict,return_dict=True)
   #valzero = {key: value.numpy() for key, value in valzero.items()}
   #valtrained= {key: value.numpy() for key, value in valtrained.items()}


   #metricsnames=alphamodel.metrics_names

   #valtrained= {metricsnames[i]: valtrained[i] for i in range(len(valtrained))}
   ##valzero = {metricsnames[i]: valzero[i] for i in range(len(valzero))}


   

   print("zero network validation loss: ")
   print(valzero)
   print("validation loss for trained network: ")
   print(valtrained)
   print("ratio of trained to zero: " + str({key + " ratio": value/(valzero[key]+1e-8) for key, value in valtrained.items()}))


   averagediscrepancyinstdevs,_=compute_transition_pointwise_measure(alphamodel,tf.cast(dataalpha["X_val"],tf.float32))
   print("average transition discrepancy in standard deviations: " + str(averagediscrepancyinstdevs))
   meanfailuretosolveequation,_,_=HYM_measure_val(alphamodel,dataalpha)
   print("mean of difference/mean of absolute value of source, weighted by sqrt(g): " + str(meanfailuretosolveequation))
   print("\n\n")
   return alphamodel,training_historyAlpha


